In [28]:
#(1)
#importing BeautifulSoup from bs4 package
from bs4 import BeautifulSoup 
#importing requests package
import requests 
#assigning the url
url = 'https://www.python.org/~guido/'

#getting the url response 
r = requests.get(url) 

#converting the response into text format
html_doc = r.text 

#parsing through the html_doc using beautifulsoup
soup = BeautifulSoup(html_doc,"html.parser")

In [29]:
#(2)
# Count the number of hyperlink URLs (which begin with http) in the <u>head</u> & <u>body</u> tags
#creating seperate bs4 element tags for head and body
hd = soup.head
bd = soup.body

#number of links is equal to the sum of links in head tag and body tag
nlinks = len(hd.find_all('a'))+len(bd.find_all('a'))
nlinks #25 zero in head + 25 in body

25

In [30]:
#(3a):  Generate a unique list of hyperlink URLs (which begin with http) from the <u>body</u> tag and convert it into a pandas <u>series</u> object.\n",
#"- If your website body has no hyperlinks, please select a new website and repeat question.

#finding all the a tags and its content and holding them in a bs.element.result:
a_rslt = bd.find_all('a')
#importing pandas as pd
import pandas as pd

#Converting list into pandas series
a_series = pd.Series(a_rslt)

#coverting series into list using tolist() method
a_list = a_series.tolist()
    

In [31]:
#(3b):  Create a function which parses the domain name (parent website) from a hyperlink URL
#function that takes a string as input and splits it into difeerent parts as specified by start and end in split function and returns the selected output
def url(s):
    s=str(s)
    domain = s.split("//")[-1].split("/")[0]
    return domain

#passing all the values in a_list to the function
a_domain = [url(d) for d in a_list]

In [32]:
# ( 3c ): Create a new pandas series composed of domain names.\n",
#    "- Consider using the <b>series.map()</b> method, applying your function from (3b) over the series in (3a)\n",

f = a_series.map(url, na_action=None)

In [40]:
### ( 4a ) Choose an RSS feed site and retrieve the XML data using urllib.\n",
 #   "- If it doesn’t return a page (perhaps because it uses cookies or other interactions to actually return content), try a different site.\n",
 #   "- Here is a list of RSS Feeds to get you started:\n",
 #   "    - <b>https://rss.com/popular-rss-feeds/</b>\n",
 #   "\n",
 #   "#### ( 4b ) Use ElementTree to parse the object (like we did in class).\n",
    
import xml.etree.ElementTree as ET 
#url = 'https://www.omnycontent.com/d/playlist/4b5f9d6d-9214-48cb-8455-a73200038129/a7c446d6-29da-43ba-bbe5-a7da00ecda4a/a65603a6-cf22-4150-91c1-a7da00ed5220/podcast.rss'
#url ='http://podcastfeeds.nbcnews.com/drone/api/query/audio/podcast/1.0/MSNBC-MTP.xml'
#url = 'http://www.hindustantimes.com/rss/topnews/rssfeed.xml'
url = 'https://feeds.a.dj.com/rss/RSSWorldNews.xml'
#url = 'https://www.w3schools.com/xml/xml_rss.asp'
#url = 'https://www.themoviedb.org/movie'
import requests
x = requests.get(url)

doc = x.text
with open('topnewsfeed.xml', 'wb') as f: 
    f.write(x.content) 
from bs4 import BeautifulSoup 
soup = BeautifulSoup(doc,'lxml')


In [38]:
# filepath = r'C:\Users\KARTHEEK SP\Desktop\topnewsfeed.xml'
# out = open(filepath, 'w')
# out.write(xmlstring)
tree = ET.parse('topnewsfeed.xml')
root = tree.getroot()

In [43]:
## ( 5 ): Count the number of Top-level <u>parent tags</u> & their number of <u>child</u> tags.\n",
   # "- Top-level means that a tag has <b><u>no</b></u> parent tag\n",
   #"- Only count direct children to these top level tags\n",
firstchild = root[0]
itemlist = firstchild.findall('item')
print("There is only", str(len(root)), "parent tag by definition")
print("There are", str(len(itemlist)), "child tags")
len(itemlist)

There is only 1 parent tag by definition
There are 7 child tags


7

In [44]:
# # ( 6a ): Convert all of the items of your RSS XML into a pandas dataframe.\n",
#     "- An item in an RSS feed begins with an item tag < item > ... < /item >\n",
#     "- Each row represents 1 item\n",
#     "- Each column represents 1 child-tag to the parent <u>item</u> tag:\n",
#     "    - i.e.: Title, Link, PubDate, Category\n",
#     "        - Ignore children of these tags\n",
import pandas as pd
import io

firstitem = firstchild.find('item')
alltitles = tree.findall('.//title')
alllinks = tree.findall('.//link')
alldescription = tree.findall('.//description')
allpubdate = tree.findall('.//pubDate')
allguid = tree.findall('.//guid')
allcategory = tree.findall('.//category')

dfcols = ['Title', 'Link', 'Description', 'Publishing Date', 'GUID', "Category"]

titles = [x.text for x in alltitles] # has the largest list, but the array must be equal in length 19
links = [x.text for x in alllinks] #19
description = [x.text for x in alldescription] #18
pubdate = [x.text for x in allpubdate] #18
guid = [x.text for x in allguid] #17
category = [x.text for x in allcategory] #17

#adjust the list to match the array values
for values in range(1):
    description.insert(values, None)
    pubdate.insert(values, None)

#adjust to match the array value 
for values in range(2):
    guid.insert(values, None)
    category.insert(values, None)


dfdata = {'Title': titles, 'Link': links, 'Description': description, 'Publishing Date': pubdate, 'GUID': guid, "Category": category}
print(dfcols)

xml_df = pd.DataFrame.from_dict(dfdata)
print(xml_df.head())

#6b .describe()
# I think this is it
xml_df.describe()


['Title', 'Link', 'Description', 'Publishing Date', 'GUID', 'Category']
                                               Title  \
0                                WSJ.com: World News   
1                                WSJ.com: World News   
2    Turkey Elections Loosen Erdogan's Grip on Power   
3  Facebook Removes Hundreds of Fake Accounts Ahe...   
4  Singapore Fake-News Bill Orders Corrections by...   

                                                Link  \
0             http://online.wsj.com/page/2_0006.html   
1             http://online.wsj.com/page/2_0006.html   
2  https://www.wsj.com/articles/erdogans-grip-on-...   
3  https://www.wsj.com/articles/facebook-removes-...   
4  https://www.wsj.com/articles/singapore-fake-ne...   

                                         Description  \
0                                               None   
1                                         World News   
2  The party of President Recep Tayyip Erdogan ap...   
3  Facebook said it has taken 

,Title,Link,Description,Publishing Date,GUID,Category
count,9,9,8,8,7,7
unique,8,8,8,8,7,1
top,WSJ.com: World News,http://online.wsj.com/page/2_0006.html,A wider range of fentanyl derivatives will be ...,"Sun, 31 Mar 2019 19:09:00 -0400",SB11709494519906504674704585215883512906228,PAID
freq,2,2,1,1,1,7
